## Descriptions

In [57]:
import pandas as pd
import numpy as np
import requests
import json
import pickle
from collections import Counter
from collections import defaultdict
from powersimdata.input.grid import Grid

### Find the county that a specific bus belongs to

In [2]:
eastern = Grid(['Eastern'])

--> Loading Eastern interconnect
Loading zone
Loading sub
Loading bus2sub
Loading bus
Loading plant
Loading plant cost
Loading branch
Loading DC line


In [4]:
# Warning: this step takes around 3 hours to finish. The results are stored in bus_ba_map.csv for future use.

bus_ba_map = eastern.bus[eastern.bus['Pd']>0][['Pd','lat','lon']].copy()
bus_ba_map.loc[:,'County'] = None
bus_ba_map.loc[:,'BA'] = None
# api-endpoint 
URL = "https://geo.fcc.gov/api/census/block/find"
# defining a params dict for the parameters to be sent to the API
bus_no_county_match = []
for index,row in bus_ba_map.iterrows():
    print(index)
    PARAMS = {'latitude':row['lat'], 'longitude':row['lon'], 'format':'json', 'showall': True} 
    # sending get request and saving the response as response object 
    r = requests.get(url = URL, params = PARAMS).json()
    try:
        county_name = r['County']['name']+'__'+r['State']['code']
        bus_ba_map.loc[index,'County'] = county_name
    except:
        bus_no_county_match.append(index) 

3
9
10
11
12
13
15
16
18
20
22
25
28
30
32
33
36
37
38
40
41
42
43
44
45
46
47
50
51
53
55
56
57
60
61
62
63
65
66
67
68
70
72
74
76
77
79
82
84
86
87
88
89
91
94
95
100
102
103
104
105
106
108
109
111
112
113
114
116
117
118
120
122
123
125
127
128
129
130
132
134
135
136
137
138
139
140
141
142
144
145
146
147
148
149
150
152
153
155
157
158
159
160
161
162
164
165
166
167
168
169
170
171
173
174
176
177
178
179
180
182
184
187
188
189
192
195
200
201
202
203
204
206
207
209
210
212
213
214
215
216
217
218
219
221
222
225
226
227
228
229
231
235
236
237
240
243
245
246
247
249
250
251
252
255
256
257
258
259
261
262
264
266
268
269
271
273
274
276
277
278
280
282
283
286
288
290
291
292
293
296
298
299
301
303
304
306
307
308
309
310
315
316
318
323
327
328
329
330
331
334
335
337
338
339
341
342
343
345
348
349
350
351
352
353
356
357
358
359
360
362
365
367
370
371
372
377
380
381
383
384
386
387
388
390
392
395
398
399
402
403
404
405
412
413
415
418
419
421
422
424
425
426
428
42

3793
3797
3801
3803
3804
3806
3807
3808
3813
3814
3816
3819
3820
3821
3822
3823
3824
3825
3826
3827
3828
3831
3837
3838
3839
3840
3841
3844
3845
3846
3847
3848
3849
3850
3852
3854
3862
3863
3864
3865
3866
3868
3869
3870
3872
3873
3876
3877
3878
3879
3881
3882
3884
3889
3892
3894
3895
3897
3899
3901
3903
3904
3905
3906
3907
3910
3914
3915
3916
3918
3919
3920
3921
3924
3926
3927
3931
3933
3938
3941
3945
3949
3952
3953
3962
3967
3974
3979
3983
3987
3994
4003
4006
4007
4015
4025
4034
4036
4037
4057
4064
4066
4073
4080
4084
4088
4092
4098
4100
4120
4494
4496
4497
4498
4499
4502
4504
4505
4506
4507
4508
4510
4512
4513
4515
4518
4520
4524
4525
4526
4527
4528
4529
4531
4533
4535
4536
4537
4538
4540
4542
4543
4545
4546
4547
4548
4550
4551
4552
4553
4555
4557
4559
4560
4562
4567
4568
4570
4572
4575
4576
4577
4578
4579
4580
4581
4582
4583
4584
4587
4589
4591
4593
4595
4600
4602
4603
4605
4607
4608
4609
4612
4613
4614
4616
4617
4620
4622
4624
4625
4627
4629
4630
4631
4632
4633
4634
4635
4637
4639


7560
7561
7562
7563
7568
7569
7571
7572
7573
7574
7575
7576
7578
7579
7580
7581
7582
7584
7586
7587
7588
7589
7590
7591
7593
7594
7595
7597
7598
7599
7600
7602
7603
7604
7605
7607
7608
7610
7611
7613
7614
7615
7618
7619
7620
7621
7623
7624
7625
7626
7627
7628
7629
7631
7632
7634
7635
7637
7638
7640
7641
7642
7643
7645
7646
7648
7649
7650
7652
7653
7654
7655
7656
7657
7659
7661
7662
7667
7669
7674
7675
7677
7678
7679
7680
7681
7682
7683
7684
7685
7686
7687
7688
7689
7694
7699
7700
7702
7703
7708
7709
7711
7712
7713
7715
7716
7718
7722
7723
7724
7726
7727
7729
7735
7736
7737
7740
7742
7745
7749
7750
7752
7753
7754
7755
7757
7760
7761
7763
7764
7765
7767
7769
7770
7771
7772
7773
7774
7775
7777
7778
7779
7780
7783
7785
7786
7787
7788
7789
7791
7792
7794
7796
7797
7798
7799
7800
7801
7803
7805
7807
7808
7809
7810
7812
7813
7815
7816
7817
7819
7820
7821
7823
7825
7826
7828
7829
7830
7831
7832
7834
7835
7838
7839
7841
7846
7847
7849
7852
7853
7854
7856
7857
7858
7861
7862
7863
7864
7865
7868


10645
10646
10648
10649
10650
10652
10653
10655
10656
10657
10658
10659
10660
10663
10664
10665
10666
10667
10668
10669
10670
10671
10672
10673
10674
10677
10679
10680
10681
10683
10684
10685
10687
10688
10689
10690
10692
10693
10695
10696
10697
10698
10699
10701
10703
10705
10706
10707
10708
10710
10712
10715
10716
10718
10719
10721
10722
10723
10724
10725
10726
10727
10729
10730
10732
10733
10734
10736
10738
10739
10740
10741
10742
10743
10748
10751
10752
10753
10756
10757
10758
10760
10761
10762
10764
10766
10773
10775
10776
10778
10779
10780
10781
10783
10785
10786
10788
10790
10792
10794
10795
10796
10797
10798
10799
10801
10803
10804
10806
10809
10811
10812
10813
10815
10816
10817
10820
10821
10822
10823
10824
10825
10830
10831
10832
10833
10836
10837
10838
10840
10842
10845
10846
10847
10848
10851
10852
10854
10855
10856
10857
10858
10859
10860
10861
10863
10864
10865
10866
10867
10868
10869
10871
10873
10874
10875
10876
10877
10879
10881
10882
10884
10886
10887
10888
10889
1089

13314
13315
13317
13318
13319
13320
13322
13323
13325
13326
13327
13329
13330
13332
13337
13339
13341
13342
13343
13344
13346
13347
13348
13349
13350
13351
13352
13353
13354
13356
13359
13360
13362
13364
13366
13367
13368
13369
13371
13372
13373
13376
13377
13378
13379
13381
13383
13384
13387
13388
13389
13390
13391
13393
13394
13395
13396
13397
13398
13400
13401
13402
13403
13404
13406
13411
13413
13414
13416
13418
13419
13421
13423
13424
13425
13426
13427
13428
13430
13431
13432
13433
13435
13436
13437
13438
13440
13441
13442
13443
13444
13447
13449
13450
13451
13453
13454
13455
13460
13461
13462
13463
13464
13465
13468
13469
13470
13471
13472
13473
13475
13476
13477
13479
13480
13481
13482
13483
13484
13486
13491
13497
13499
13501
13502
13504
13505
13506
13508
13510
13511
13513
13514
13515
13516
13518
13519
13520
13521
13525
13527
13529
13533
13535
13537
13539
13541
13542
13545
13546
13548
13549
13550
13551
13552
13555
13556
13557
13563
13564
13565
13567
13568
13569
13571
13572
1357

...

56589
56590
56591
56593
56594
56595
56597
56598
56600
56601
56602
56603
56604
56606
56607
56608
56609
56611
56612
56613
56614
56615
56616
56619
56621
56625
56626
56631
56632
56633
56635
56638
56641
56642
56647
56650
56651
56654
56658
56660
56661
56663
56666
56668
56671
56672
56673
56674
56677
56680
56681
56683
56685
56686
56689
56691
56693
57021
57022
57024
57026
57029
57030
57031
57033
57037
57038
57041
57045
57047
57048
57049
57050
57051
57052
57056
57058
57059
57060
57062
57064
57066
57067
57068
57070
57071
57072
57074
57081
57083
57085
57088
57090
57091
57093
57097
57101
57102
57105
57108
57109
57110
57111
57112
57113
57116
57119
57121
57122
57124
57125
57126
57127
57128
57129
57130
57131
57133
57136
57137
57139
57141
57143
57145
57147
57148
57151
57152
57155
57157
57159
57160
57161
57163
57164
57167
57168
57171
57172
57174
57176
57180
57181
57182
57183
57186
57187
57188
57189
57194
57196
57199
57200
57202
57203
57204
57205
57206
57209
57212
57215
57219
57223
57224
57225
57226
5722

59696
59698
59699
59700
59702
59703
59948
59949
59950
59951
59952
59953
59955
59958
59960
59962
59963
59964
59966
59967
59968
59969
59970
59971
59972
59974
59975
59976
59978
59979
59980
59982
59984
59985
59987
59988
59989
59990
59992
59996
59997
59998
59999
60000
60001
60003
60004
60005
60006
60007
60008
60010
60011
60012
60013
60014
60017
60018
60019
60020
60021
60023
60024
60026
60028
60029
60030
60031
60032
60034
60035
60036
60038
60039
60040
60041
60043
60049
60054
60056
60057
60059
60061
60062
60064
60066
60067
60068
60069
60071
60072
60078
60079
60081
60082
60083
60084
60085
60087
60089
60090
60093
60095
60096
60099
60104
60106
60107
60110
60111
60112
60113
60114
60116
60117
60119
60121
60122
60123
60125
60126
60127
60129
60131
60132
60133
60134
60135
60136
60139
60140
60141
60142
60143
60145
60146
60147
60148
60149
60150
60151
60152
60153
60156
60157
60158
60159
60161
60166
60168
60170
60172
60173
60174
60175
60176
60177
60178
60179
60182
60184
60185
60186
60188
60189
60190
6019

62568
62569
62571
62572
62574
62575
62577
62579
62581
62582
62583
62584
62585
62586
62587
62588
62589
62590
62593
62594
62595
62597
62598
62600
62601
62602
62603
62605
62606
62607
62608
62610
62612
62613
62615
62616
62618
62620
62622
62624
62625
62626
62627
62628
62630
62632
62633
62638
62640
62641
62642
62644
62645
62646
62648
62649
62651
62652
62653
62655
62657
62658
62659
62661
62662
62663
62664
62665
62667
62668
62669
62670
62671
62672
62674
62676
62677
62678
62680
62681
62682
62683
62684
62686
62687
62688
62690
62691
62692
62693
62694
62695
62696
62698
62700
62701
62702
62703
62704
62705
62706
62707
62709
62710
62712
62714
62715
62716
62717
62718
62720
62722
62723
62724
62726
62727
62729
62730
62733
62734
62735
62738
62739
62740
62741
62742
62743
62744
62745
62746
62747
62748
62749
62750
62752
62753
62754
62755
62756
62758
62759
62760
62761
62762
62763
62764
62765
62766
62769
62771
62772
62773
62774
62776
62777
62778
62780
62781
62783
62785
62786
62787
62788
62789
62790
62792
6279

66203
66205
66210
66213
66218
66220
66225
66227
66230
66232
66237
66238
66240
66241
66246
66247
66249
66251
66252
66255
66260
66261
66263
66266
66271
66273
66275
66277
66278
66279
66281
66283
66285
66286
66287
66289
66290
66291
66293
66294
66296
66299
66302
66305
66306
66308
66314
66316
66317
66322
66323
66324
66326
66327
66328
66331
66332
66335
66337
66340
66342
66347
66348
66349
66350
66352
66355
66358
66359
66360
66361
66363
66364
66365
66366
66367
66370
66372
66374
66375
66378
66381
66382
66385
66387
66388
66389
66390
66392
66397
66399
66400
66401
66403
66404
66407
66409
66410
66411
66412
66413
66414
66415
66416
66417
66419
66421
66426
66429
66431
66432
66433
66434
66435
66436
66438
66441
66442
66446
66448
66452
66454
66455
66456
66459
66462
66464
66465
66470
66471
66472
66477
66479
66480
66481
66482
66484
66485
66486
66487
66488
66491
66492
66493
66494
66497
66498
66499
66500
66501
66503
66506
66510
66511
66512
66515
66517
66519
66521
66523
66524
66525
66526
66528
66530
66533
6653

69458
69461
69463
69464
69465
69466
69468
69469
69470
69471
69472
69474
69476
69478
69480
69483
69485
69491
69494
69495
69497
69500
69502
69503
69504
69505
69506
69507
69508
69510
69511
69514
69516
69518
69522
69523
69524
69527
69528
69530
69532
69533
69534
69535
69536
69539
69540
69541
69544
69546
69549
69551
69554
69555
69557
69558
69559
69560
69561
69564
69569
69572
69575
69578
69581
69582
69583
69584
69586
69587
69588
69589
69592
69597
69599
69601
69763
69764
69767
69770
69771
69772
69775
69780
69783
69784
69785
69787
69789
69791
69793
69794
69800
69803
69804
69805
69807
69808
69809
69811
69817
69819
69821
69824
69825
69826
69829
69830
69831
69832
69833
69834
69835
69840
69842
69844
69846
69848
69851
69852
69853
69855
69857
69858
69859
69860
69863
69864
69870
69873
69874
69876
69877
69878
69879
69880
69881
69882
69885
69890
69892
69893
69895
69898
69900
69901
69902
69907
69910
69915
69919
69922
69923
69924
69926
69930
69931
69932
69933
69934
69939
69940
69942
69943
69946
69948
6994

### Find the BA that a specific bus belongs to based on the county

In [3]:
data = json.load(open('EasternBAtoCountyDraft6.txt'))
# df = pd.DataFrame(data['groups']['#cc3333']['paths'])
bus_ba_map = pd.read_csv('bus_ba_map.csv',index_col=0)

In [4]:
ba_county_list = {}
for val in data['groups'].values():
    ba_county_list[val['label']] = set(val['paths'])

In [5]:
for index,row in bus_ba_map.iterrows():
    for BA,clist in ba_county_list.items():
        try:
            county = row['County'].replace(' ','_')
            county = county.replace('.','')
            county = county.replace('-','')
            county = county.replace('\'','_')
            if row['County'] == 'LaSalle__IL':
                county = 'La_Salle__IL'
            if row['County'] == 'Lac Qui Parle__MN':
                county = 'Lac_qui_Parle__MN'
            if row['County'] == 'Baltimore__MD':
                county = 'Baltimore_County__MD'
            if row['County'] == 'District of Columbia__DC':
                county = 'Washington__DC'
            if row['County'] == 'St. Louis City__MO':
                county = 'St_Louis_Co__MO'
            if county in clist:
                bus_ba_map.loc[index,'BA'] = BA
                break
        except:
            continue

# bus_no_BA_match = list(bus_ba_map[~bus_ba_map['BA'].astype(bool)].index)
bus_no_BA_match = list(bus_ba_map[bus_ba_map['BA'].isna()].index)
bus_no_county_match = list(bus_ba_map[bus_ba_map['County'].isna()].index)

# Add zone name into the data frame for reference
bus_ba_map.loc[:,'zone_name'] = eastern.bus[eastern.bus['Pd']>0]['zone_id'].apply(lambda x: eastern.id2zone[x])

# Fix mismatch county names in Virginia and Maryland
for ind in bus_no_BA_match:
    if bus_ba_map.loc[ind,'zone_name'] in {'Virginia Mountains','West Virginia','Virginia Tidewater','Maryland'}:
        bus_ba_map.loc[ind,'BA'] = 'PJM'
        
# Manually assign outliers (outside US territory) to the nearest BA
# bus no county match: [91: ISNE, 7991: NYIS, 7992: NYIS, 8707: NYIS, 8708: NYIS, 40644: MISO]
bus_ba_map.loc[91,'BA'] = 'ISNE'
bus_ba_map.loc[7991,'BA'] = 'NYIS'
bus_ba_map.loc[7992,'BA'] = 'NYIS'
bus_ba_map.loc[8707,'BA'] = 'NYIS'
bus_ba_map.loc[8708,'BA'] = 'NYIS'
bus_ba_map.loc[40644,'BA'] = 'MISO'

# Assign the rest no-ba-match buses to SWPP
for ind in bus_no_BA_match:
    bus_ba_map.loc[ind,'BA'] = 'SWPP'

# Assign buses in ERCOT Texas to SWPP or MISO based on the location by observation
miso_tx_ind = bus_ba_map[(bus_ba_map['BA']=='ERCOT Texas') & (bus_ba_map['zone_name']=='East Texas') & ((bus_ba_map['County']=='Montgomery__TX') | (bus_ba_map['County']=='Walker__TX'))].index
for ind in bus_ba_map[bus_ba_map['BA']=='ERCOT Texas'].index:
    if ind in miso_tx_ind:
        bus_ba_map.loc[ind,'BA']='MISO'
    else:
        bus_ba_map.loc[ind,'BA']='SWPP'

In [6]:
# Make BA Code consistent with EIA data source
ba_code_fix = {'ISONE':'ISNE','NYISO':'NYIS'}
bus_ba_map.replace(ba_code_fix,inplace=True)

In [7]:
ba_county_list.keys()

dict_keys(['ISONE', 'NYISO', 'PJM', 'AEC', 'SOCO', 'TVA_LGEE', 'Florida', 'MISO', 'Carolina', 'ERCOT Texas'])

In [8]:
bus_ba_map.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 32460 entries, 3 to 69953
Data columns (total 6 columns):
Pd           32460 non-null float64
lat          32460 non-null float64
lon          32460 non-null float64
County       32454 non-null object
BA           32460 non-null object
zone_name    32460 non-null object
dtypes: float64(3), object(3)
memory usage: 3.0+ MB


In [62]:
bus_ba_map.to_csv('bus_ba_map.csv')

In [54]:
# Assign a list of BA into BA areas we defined above
BA_area ={
    'ISNE':['ISNE'],
    'NYIS':['NYIS'],
    'PJM':['PJM','OVEC'],
    'AEC':['AEC'],
    'SOCO':['SOCO'],
    'TVA_LGEE':['TVA','LGEE'],
    'Florida':['TAL','JEA','GVL','SEC','FPC','NSB','FMPP','TEC','FPL','HST'],
    'MISO':['MISO','AECI'],
    'SWPP':['SWPP','SPA'],
    'Carolina':['CPLW','DUK','SCEG','SC','CPLE']
}

In [59]:
with open('BA_area.pkl', 'wb') as f:
    pickle.dump(BA_area, f, protocol=pickle.HIGHEST_PROTOCOL)

### Plot check

In [10]:
from bokeh.io import show,output_notebook,output_file,export
from bokeh.models.widgets import Panel, Tabs
from bokeh.models import (
    ColumnDataSource,
    HoverTool,
    LogColorMapper,
    CustomJS,
    Slider,
    CheckboxGroup,
    Select,
    Div,
    TapTool,
    ColorBar,
    LinearColorMapper
)
from bokeh.palettes import Viridis6 as palette
from bokeh.plotting import figure
from bokeh.layouts import (widgetbox, row, gridplot)

# from bokeh.tile_providers import CARTODBPOSITRON,STAMEN_TONER_BACKGROUND,STAMEN_TONER,STAMEN_TERRAIN

# import bokeh.sampledata
# bokeh.sampledata.download()
from bokeh.sampledata.us_counties import data as counties
from bokeh.sampledata import us_states
from bokeh import events

import matplotlib.pyplot as plt
from bokeh.colors.groups import yellow
import bokeh.colors

import seaborn as sns
import matplotlib.cm as cm
import matplotlib

states = us_states.data.copy()

del states["HI"]
del states["AK"]

state_xs = [states[code]["lons"] for code in states]
state_ys = [states[code]["lats"] for code in states]

us_counties = counties.copy()

counties_xs = [us_counties[code]["lons"] for code in us_counties if us_counties[code]["state"] not in {"ak", "hi", "pr", "gu", "vi", "mp", "as"}]
counties_ys = [us_counties[code]["lats"] for code in us_counties if us_counties[code]["state"] not in {"ak", "hi", "pr", "gu", "vi", "mp", "as"}]


In [11]:
# Plot for buses outside the US territory
source = eastern.bus.loc[list(bus_ba_map[bus_ba_map['County'].isna()].index)]
source.loc[:,'area_color'] = source['zone_id'].apply(lambda x: bokeh.palettes.Category20[20][(x-1)%20])

p = figure(title="", 
           toolbar_location="left", plot_width=1300, plot_height=700)

p.patches(state_xs, state_ys, fill_alpha=0,
    line_color="gray", line_alpha=0.5, line_width=1.0)
p.patches(counties_xs, counties_ys, fill_alpha=0,
    line_color="gray", line_alpha=0.5, line_width=0.5)

p.scatter(source=source,x='lon',y='lat',size=10,marker='o',color='area_color')

show(p)

In [12]:
# Plot for buses no ba match
source = eastern.bus.loc[list(bus_ba_map[bus_ba_map['BA'].isna()].index)]
source.loc[:,'area_color'] = source['zone_id'].apply(lambda x: bokeh.palettes.Category20[20][(x-1)%20])

p = figure(title="", 
           toolbar_location="left", plot_width=1300, plot_height=700)

p.patches(state_xs, state_ys, fill_alpha=0,
    line_color="gray", line_alpha=0.5, line_width=1.0)
p.patches(counties_xs, counties_ys, fill_alpha=0,
    line_color="gray", line_alpha=0.5, line_width=0.5)

p.scatter(source=source,x='lon',y='lat',size=2,marker='o',color='area_color')

show(p)

In [46]:
# Plot for buses assigned to a specific BA area
source = eastern.bus.loc[list(bus_ba_map[bus_ba_map['BA']=='MISO'].index)]
source.loc[:,'area_color'] = source['zone_id'].apply(lambda x: bokeh.palettes.Category20[20][(x-1)%20])

p = figure(title="", 
           toolbar_location="left", plot_width=1300, plot_height=700)

p.patches(state_xs, state_ys, fill_alpha=0,
    line_color="gray", line_alpha=0.5, line_width=1.0)
p.patches(counties_xs, counties_ys, fill_alpha=0,
    line_color="gray", line_alpha=0.5, line_width=0.5)

p.scatter(source=source,x='lon',y='lat',size=2,marker='o',color='area_color')

show(p)